
# NBA Predictor
### By Isaac Tabb

## Setup

### Hide Warnings

In [8]:
import warnings
warnings.filterwarnings('ignore')

### Define league

#### Only do these once

In [9]:
# define the nba teams overall ratings
nba_teams = {
    "East": {'BOS':70, 'CLE':61, 'MIL':69, 'PHI':65, 'BKN':50, 'NYK':53, 'ATL':50, 'IND':46, 'TOR':46, 'MIA':54, 'WAS':45, 'ORL':41, 'CHI':49, 'CHA':27, 'DET':27},
    "West": {'DAL':57, 'SAC':57, 'MEM':60, 'DEN':69, 'MIN':52, 'PHX':59, 'LAC':54, 'GSW':52, 'NOP':51, 'UTA':49, 'POR':48, 'OKC':47, 'LAL':46, 'SAS':26, 'HOU':24}
    }

longevity = {
    "East": {'BOS':4, 'CLE':4, 'MIL':3, 'PHI':2, 'BKN':3, 'NYK':-2, 'ATL':0, 'IND':2, 'TOR':-2, 'MIA':-4, 'WAS':-4, 'ORL':3, 'CHI':-4, 'CHA':-3, 'DET':4},
    "West": {'DAL':2, 'SAC':2, 'MEM':4, 'DEN':2, 'MIN':-3, 'PHX':-2, 'LAC':-3, 'GSW':-2, 'NOP':3, 'UTA':3, 'POR':-4, 'OKC':4, 'LAL':-5, 'SAS':-2, 'HOU':3}
}

season = 2022

In [10]:
# define league overall scores
import pandas as pd

east_ovr = pd.DataFrame(columns=["TM", "Champ", "Champ App.", "Playoff App."])
west_ovr = pd.DataFrame(columns=["TM", "Champ", "Champ App.", "Playoff App."])

# add teams to respective data frames
for team in nba_teams["East"]:
  new_row = {"TM":team, "Champ":0, "Champ App.":0, "Playoff App.":0}
  east_ovr = east_ovr.append(new_row, ignore_index=True)

for team in nba_teams["West"]:
  new_row = {"TM":team, "Champ":0, "Champ App.":0, "Playoff App.":0}
  west_ovr = west_ovr.append(new_row, ignore_index=True)

# reset indexes in dataframe
import numpy as np
east_ovr.index = np.arange(1, len(east_ovr) + 1)
west_ovr.index = np.arange(1, len(west_ovr) + 1)

#### Start League Block

In [11]:
# create the league dataframes
import pandas as pd

east = pd.DataFrame(columns=["TM", "W", "L", "%", "Past10", "Strk"])
west = pd.DataFrame(columns=["TM", "W", "L", "%", "Past10", "Strk"])

# add teams to respective data frames
for team in nba_teams["East"]:
  new_row = {"TM":team, "W":0, "L":0, "%":0.0, "Past10":'0-0', "Strk":'NA'}
  east = east.append(new_row, ignore_index=True)

for team in nba_teams["West"]:
  new_row = {"TM":team, "W":0, "L":0, "%":0.0, "Past10":'0-0', "Strk":'NA'}
  west = west.append(new_row, ignore_index=True)

# reset indexes in dataframe
import numpy as np
east.index = np.arange(1, len(east) + 1)
west.index = np.arange(1, len(west) + 1)

# define past 10 games dictionary
nba_teams_p10 = {'East':{}, 'West':{}}
for team in nba_teams['East']:
  nba_teams_p10['East'][team] = []

for team in nba_teams['West']:
  nba_teams_p10['West'][team] = []

### Predict Game Function

In [12]:
import random
def predict_game(nba_teams, nba_teams_p10, east, west):
  # iterate thru east teams
  for team in nba_teams['East']:
    # this function makes it so that every game:
    #   (1) good teams have a 1% chance of getting  worse
    #   (2) mid teams have a 1% chance of getting worse or better
    #   (3) bad teams have a 1% chance of getting better
    # if team is amongst the "good range" (above 60)
    if nba_teams['East'].get(team) >= 60:
      # random possible team setback
      rand_team_setback_val = random.randint(0,100)
      # the 'death' value is 69
      if rand_team_setback_val == 69:
        setback_amt = random.randint(3,10)
        nba_teams['East'][team] = nba_teams['East'][team] - setback_amt
    elif (nba_teams['East'].get(team) > 40 and nba_teams['East'].get(team) < 60):
      # random possible team setback
      rand_team_setback_val = random.randint(0,100)
      # the 'life or death' value is 69
      if rand_team_setback_val == 69:
        setback_amt = random.randint(-10,10)
        nba_teams['East'][team] = nba_teams['East'][team] - setback_amt
    elif nba_teams['East'].get(team) <= 40:
      # random possible team setback
      rand_team_setback_val = random.randint(0,100)
      # the 'life' value is 69
      if rand_team_setback_val == 69:
        setback_amt = random.randint(3,10)
        nba_teams['East'][team] = nba_teams['East'][team] + setback_amt

    # pick a random value for win or loss
    rand_val = random.randint(0,100)
    if rand_val > nba_teams['East'].get(team):
      # increment loss
      east['L'][east.index[east['TM']==team]] += 1

      # adjust the teams past 10 record
      if len(nba_teams_p10['East'][team]) == 10:
        del nba_teams_p10['East'][team][0]
      nba_teams_p10['East'][team].append('L')
      p10w = str(nba_teams_p10['East'][team].count('W'))
      p10l = str(nba_teams_p10['East'][team].count('L'))
      east["Past10"][east.index[east['TM']==team]] = p10w + '-' + p10l

      # adjust team's streak
      strk = east["Strk"].values[east.index[east['TM']==team]-1]
      if strk[0][0] == 'L':
        if strk[0][1] == '3':
          nba_teams['East'][team] -= 1
        elif strk[0][1] == '9':
          nba_teams['East'][team] -= 1
        east["Strk"][east.index[east['TM']==team]] = 'L' + str(int(strk[0][1:])+1)
      else:
        east["Strk"][east.index[east['TM']==team]] = 'L1'

    # else is win, same thing
    else:
      east['W'][east.index[east['TM']==team]] += 1

      if len(nba_teams_p10['East'][team]) == 10:
        del nba_teams_p10['East'][team][0]
      nba_teams_p10['East'][team].append('W')
      p10w = str(nba_teams_p10['East'][team].count('W'))
      p10l = str(nba_teams_p10['East'][team].count('L'))
      east["Past10"][east.index[east['TM']==team]] = p10w + '-' + p10l

      strk = east["Strk"].values[east.index[east['TM']==team]-1]
      if strk[0][0] == 'W':
        if strk[0][1] == '3':
          nba_teams['East'][team] += 1
        elif strk[0][1] == '9':
          nba_teams['East'][team] += 1
        east["Strk"][east.index[east['TM']==team]] = 'W' + str(int(strk[0][1:])+1)
      else:
        east["Strk"][east.index[east['TM']==team]] = 'W1'

    # adjust win percentage
    east['%'][east.index[east['TM']==team]] = east['W'][east.index[east['TM']==team]]/(east['W'][east.index[east['TM']==team]]+east['L'][east.index[east['TM']==team]])
    east['%'][east.index[east['TM']==team]] = east['%'][east.index[east['TM']==team]].astype(float).round(2)

  # same thing for west
  for team in nba_teams['West']:

    if nba_teams['West'].get(team) >= 60:
      # random possible team setback
      rand_team_setback_val = random.randint(0,100)
      # the 'death' value is 69
      if rand_team_setback_val == 69:
        setback_amt = random.randint(3,10)
        nba_teams['West'][team] = nba_teams['West'][team] - setback_amt
    elif (nba_teams['West'].get(team) > 40 and nba_teams['West'].get(team) < 60):
      # random possible team setback
      rand_team_setback_val = random.randint(0,100)
      # the 'life or death' value is 69
      if rand_team_setback_val == 69:
        setback_amt = random.randint(-10,10)
        nba_teams['West'][team] = nba_teams['West'][team] - setback_amt
    elif nba_teams['West'].get(team) <= 40:
      # random possible team setback
      rand_team_setback_val = random.randint(0,100)
      # the 'life' value is 69
      if rand_team_setback_val == 69:
        setback_amt = random.randint(3,10)
        nba_teams['West'][team] = nba_teams['West'][team] + setback_amt


    rand_val = random.randint(0,100)
    if rand_val > nba_teams['West'].get(team):
      west['L'][west.index[west['TM']==team]] += 1

      if len(nba_teams_p10['West'][team]) == 10:
        del nba_teams_p10['West'][team][0]
      nba_teams_p10['West'][team].append('L')
      p10w = str(nba_teams_p10['West'][team].count('W'))
      p10l = str(nba_teams_p10['West'][team].count('L'))
      west["Past10"][west.index[west['TM']==team]] = p10w + '-' + p10l

      strk = west["Strk"].values[west.index[west['TM']==team]-1]
      if strk[0][0] == 'L':
        if strk[0][1] == '3':
          nba_teams['West'][team] -= 1
        elif strk[0][1] == '9':
          nba_teams['West'][team] -= 1
        west["Strk"][west.index[west['TM']==team]] = 'L' + str(int(strk[0][1:])+1)
      else:
        west["Strk"][west.index[west['TM']==team]] = 'L1'
    else:
      west['W'][west.index[west['TM']==team]] += 1

      if len(nba_teams_p10['West'][team]) == 10:
        del nba_teams_p10['West'][team][0]
      nba_teams_p10['West'][team].append('W')
      p10w = str(nba_teams_p10['West'][team].count('W'))
      p10l = str(nba_teams_p10['West'][team].count('L'))
      west["Past10"][west.index[west['TM']==team]] = p10w + '-' + p10l

      strk = west["Strk"].values[west.index[west['TM']==team]-1]
      if strk[0][0] == 'W':
        if strk[0][1] == '3':
          nba_teams['West'][team] += 1
        elif strk[0][1] == '9':
          nba_teams['West'][team] += 1
        west["Strk"][west.index[west['TM']==team]] = 'W' + str(int(strk[0][1:])+1)
      else:
        west["Strk"][west.index[west['TM']==team]] = 'W1'

    west['%'][west.index[west['TM']==team]] = west['W'][west.index[west['TM']==team]]/(west['W'][west.index[west['TM']==team]]+west['L'][west.index[west['TM']==team]])
    west['%'][west.index[west['TM']==team]] = west['%'][west.index[west['TM']==team]].astype(float).round(2)

    # adjustment to keep teams from getting overpowered during season
    for team in nba_teams['East']:
      if nba_teams['East'][team] < 32:
        nba_teams['East'][team] = 32
      elif nba_teams['East'][team] > 68:
        nba_teams['East'][team] = 68

    for team in nba_teams['West']:
      if nba_teams['West'][team] < 32:
        nba_teams['West'][team] = 32
      elif nba_teams['West'][team] > 68:
        nba_teams['West'][team] = 68


## Season Simulator

Use this to simulate games.

In [13]:
from tqdm import tqdm
for i in tqdm(range(82)):
  predict_game(nba_teams, nba_teams_p10, east, west)
east = east.sort_values(by=["%", "Strk"], ascending=False)
west = west.sort_values(by=["%", "Strk"], ascending=False)
east = east.reset_index(drop=True)
west = west.reset_index(drop=True)
east.index = np.arange(1, len(east) + 1)
west.index = np.arange(1, len(west) + 1)

100%|██████████| 82/82 [00:19<00:00,  4.12it/s]


Print the east/west standings and the team overalls.

In [14]:
east

,TM,W,L,%,Past10,Strk
1,MIL,60,22,0.73,8-2,W2
2,CLE,53,29,0.65,3-7,L3
3,BOS,53,29,0.65,7-3,L1
4,MIA,49,33,0.6,6-4,L2
5,PHI,47,35,0.57,6-4,W2
6,CHI,45,37,0.55,6-4,L1
7,NYK,44,38,0.54,5-5,W4
8,DET,43,39,0.52,8-2,W5
9,IND,43,39,0.52,5-5,L1
10,BKN,42,40,0.51,5-5,W1


In [15]:
west

,TM,W,L,%,Past10,Strk
1,DEN,58,24,0.71,8-2,W6
2,PHX,55,27,0.67,7-3,W4
3,SAC,54,28,0.66,8-2,W2
4,MEM,50,32,0.61,9-1,W7
5,DAL,50,32,0.61,7-3,W3
6,MIN,45,37,0.55,4-6,L3
7,GSW,45,37,0.55,6-4,L1
8,NOP,44,38,0.54,4-6,L2
9,OKC,43,39,0.52,6-4,W1
10,LAL,39,43,0.48,9-1,W4


#### Save the season standings to CSV

In [16]:
east.to_csv('season' + str(season) + 'east.csv')
west.to_csv('season' + str(season) + 'west.csv')

## Playoffs

### Playoff Functions

##### Define Playoff Teams

In [17]:
playoff_teams = {"East":{}, "West":{}}
playoff_results = pd.DataFrame(columns=["Round", "Conf", "Winner", "Loser", "Score"])

def get_playoff_teams(playoff_teams, nba_teams, east, west):
  for index, row in east.iterrows():
    if index <= 10:
      playoff_teams["East"][index] = {"Team": row['TM'], "Rtg": nba_teams['East'].get(row['TM'])}

  for index, row in west.iterrows():
    if index <= 10:
      playoff_teams["West"][index] = {"Team": row['TM'], "Rtg": nba_teams['West'].get(row['TM'])}

  return playoff_teams

playoff_teams = get_playoff_teams(playoff_teams, nba_teams, east, west)

In [18]:
playoff_teams

{'East': {1: {'Team': 'MIL', 'Rtg': 68},
  2: {'Team': 'CLE', 'Rtg': 66},
  3: {'Team': 'BOS', 'Rtg': 68},
  4: {'Team': 'MIA', 'Rtg': 60},
  5: {'Team': 'PHI', 'Rtg': 58},
  6: {'Team': 'CHI', 'Rtg': 54},
  7: {'Team': 'NYK', 'Rtg': 52},
  8: {'Team': 'DET', 'Rtg': 50},
  9: {'Team': 'IND', 'Rtg': 47},
  10: {'Team': 'BKN', 'Rtg': 47}},
 'West': {1: {'Team': 'DEN', 'Rtg': 68},
  2: {'Team': 'PHX', 'Rtg': 60},
  3: {'Team': 'SAC', 'Rtg': 64},
  4: {'Team': 'MEM', 'Rtg': 63},
  5: {'Team': 'DAL', 'Rtg': 60},
  6: {'Team': 'MIN', 'Rtg': 55},
  7: {'Team': 'GSW', 'Rtg': 55},
  8: {'Team': 'NOP', 'Rtg': 53},
  9: {'Team': 'OKC', 'Rtg': 60},
  10: {'Team': 'LAL', 'Rtg': 47}}}

##### Predict Play-In Game Function

In [19]:
def play_in_predict_matchup(team1, team2, nba_teams, matchup_teams, conf):
  rand_val = random.randint(0,nba_teams[conf][matchup_teams[conf][team1]] + nba_teams[conf][matchup_teams[conf][team2]])
  if rand_val < (nba_teams[conf][matchup_teams[conf][team1]] + 10):
    return team1, team2
  else:
    return team2, team1

##### Play-In Function

In [20]:
def play_in(playoff_teams, nba_teams, east, west, east_ovr, west_ovr, playoff_results):
  play_in_teams = {"East":{}, "West":{}}
  for seed in range(7,11):
    play_in_teams["East"][seed] = playoff_teams['East'][seed]['Team']

  for seed in range(7,11):
    play_in_teams["West"][seed] = playoff_teams['West'][seed]['Team']

  # Eastern Play In Tournament Display
  print("Eastern Conf. Play-In Tournament")
  print('(7) ' + play_in_teams['East'][7] + ' vs. ' + play_in_teams['East'][8] + ' (8)')
  print('(9) ' + play_in_teams['East'][9] + ' vs. ' + play_in_teams['East'][10] + ' (10)')
  input("Press enter to simulate: ")
  print()

  # predict 7 vs 10
  sev_eig_win, sev_eig_loss = play_in_predict_matchup(7, 8, nba_teams, play_in_teams, "East")
  # predict 8 vs 9
  nine_ten_win, nine_ten_loss = play_in_predict_matchup(9, 10, nba_teams, play_in_teams, "East")

  # display winners
  print("Eastern Conf. Play-In Rd. 1 Results")
  print(play_in_teams['East'][sev_eig_win] + ' defeats ' + play_in_teams['East'][sev_eig_loss])
  winner_str = '(' + str(sev_eig_win) + ') ' + play_in_teams['East'][sev_eig_win]
  loser_str = '(' + str(sev_eig_loss) + ') ' + play_in_teams['East'][sev_eig_loss]
  new_row = {"Round":'Play-In', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":'1-0'}
  playoff_results = playoff_results.append(new_row, ignore_index=True)

  print(play_in_teams['East'][nine_ten_win] + ' defeats ' + play_in_teams['East'][nine_ten_loss])
  winner_str = '(' + str(nine_ten_win) + ') ' + play_in_teams['East'][nine_ten_win]
  loser_str = '(' + str(nine_ten_loss) + ') ' + play_in_teams['East'][nine_ten_loss]
  new_row = {"Round":'Play-In', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":'1-0'}
  playoff_results = playoff_results.append(new_row, ignore_index=True)

  print(play_in_teams['East'][sev_eig_win] + ' makes the playoffs')
  print(play_in_teams['East'][nine_ten_loss] + ' is eliminated')
  input("Press enter to proceed: ")

  # remove loser of 9/10 and winner of 7/8 from play in
  del play_in_teams['East'][nine_ten_loss]
  del play_in_teams['East'][sev_eig_win]
  # save the seven seed
  seven_seed = playoff_teams['East'].get(sev_eig_win)
  # delete seeds from playoffs
  del playoff_teams['East'][sev_eig_win]
  del playoff_teams['East'][nine_ten_loss]

  print()
  print("Eastern Conf. Play-In Tournament Final Game")
  print('(' + str(sev_eig_loss) + ') ' + play_in_teams['East'][sev_eig_loss] + ' vs. '
        + play_in_teams['East'][nine_ten_win] + ' (' + str(nine_ten_win) + ')')
  input("Press enter to simulate: ")
  print()

  # predict final game
  final_game_win, final_game_loss = play_in_predict_matchup(sev_eig_loss, nine_ten_win, nba_teams, play_in_teams, "East")
  print("Eastern Conf. Play-In Rd. 2 Results")
  print(play_in_teams['East'][final_game_win] + ' defeats ' + play_in_teams['East'][final_game_loss])
  winner_str = '(' + str(final_game_win) + ') ' + play_in_teams['East'][final_game_win]
  loser_str = '(' + str(final_game_loss) + ') ' + play_in_teams['East'][final_game_loss]
  new_row = {"Round":'Play-In', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":'1-0'}
  playoff_results = playoff_results.append(new_row, ignore_index=True)

  print(play_in_teams['East'][final_game_win] + ' makes the playoffs')
  print(play_in_teams['East'][final_game_loss] + ' is eliminated')
  input("Press enter to proceed: ")
  print()

  # remove loser
  del play_in_teams['East'][final_game_win]
  del play_in_teams['East'][final_game_loss]
  # save eight seed
  eight_seed = playoff_teams['East'].get(final_game_win)
  # remove from playoffs
  del playoff_teams['East'][final_game_win]
  del playoff_teams['East'][final_game_loss]
  # reset seven eight
  playoff_teams['East'][7] = seven_seed
  playoff_teams['East'][8] = eight_seed

  ###############

  # Western Play In Tournament Display
  print("Western Conf. Play-In Tournament")
  print('(7) ' + play_in_teams['West'][7] + ' vs. ' + play_in_teams['West'][8] + ' (8)')
  print('(9) ' + play_in_teams['West'][9] + ' vs. ' + play_in_teams['West'][10] + ' (10)')
  input("Press enter to simulate: ")
  print()

  # predict 7 vs 10
  sev_eig_win, sev_eig_loss = play_in_predict_matchup(7, 8, nba_teams, play_in_teams, "West")
  # predict 8 vs 9
  nine_ten_win, nine_ten_loss = play_in_predict_matchup(9, 10, nba_teams, play_in_teams, "West")

  # display winners
  print("Western Conf. Play-In Rd. 1 Results")
  print(play_in_teams['West'][sev_eig_win] + ' defeats ' + play_in_teams['West'][sev_eig_loss])
  winner_str = '(' + str(sev_eig_win) + ') ' + play_in_teams['West'][sev_eig_win]
  loser_str = '(' + str(sev_eig_loss) + ') ' + play_in_teams['West'][sev_eig_loss]
  new_row = {"Round":'Play-In', "Conf":"West", "Winner":winner_str, "Loser":loser_str, "Score":'1-0'}
  playoff_results = playoff_results.append(new_row, ignore_index=True)

  print(play_in_teams['West'][nine_ten_win] + ' defeats ' + play_in_teams['West'][nine_ten_loss])
  winner_str = '(' + str(nine_ten_win) + ') ' + play_in_teams['West'][nine_ten_win]
  loser_str = '(' + str(nine_ten_loss) + ') ' + play_in_teams['West'][nine_ten_loss]
  new_row = {"Round":'Play-In', "Conf":"West", "Winner":winner_str, "Loser":loser_str, "Score":'1-0'}
  playoff_results = playoff_results.append(new_row, ignore_index=True)

  print(play_in_teams['West'][sev_eig_win] + ' makes the playoffs')
  print(play_in_teams['West'][nine_ten_loss] + ' is eliminated')
  input("Press enter to proceed: ")

  # remove loser of 9/10 and winner of 7/8 from play in
  del play_in_teams['West'][nine_ten_loss]
  del play_in_teams['West'][sev_eig_win]
  # save the seven seed
  seven_seed = playoff_teams['West'].get(sev_eig_win)
  # delete seeds from playoffs
  del playoff_teams['West'][sev_eig_win]
  del playoff_teams['West'][nine_ten_loss]

  print()
  print("Western Conf. Play-In Tournament Final Game")
  print('(' + str(sev_eig_loss) + ') ' + play_in_teams['West'][sev_eig_loss] + ' vs. '
        + play_in_teams['West'][nine_ten_win] + ' (' + str(nine_ten_win) + ')')
  input("Press enter to simulate: ")
  print()

  # predict final game
  final_game_win, final_game_loss = play_in_predict_matchup(sev_eig_loss, nine_ten_win, nba_teams, play_in_teams, "West")
  print("Western Conf. Play-In Rd. 2 Results")
  print(play_in_teams['West'][final_game_win] + ' defeats ' + play_in_teams['West'][final_game_loss])
  winner_str = '(' + str(final_game_win) + ') ' + play_in_teams['West'][final_game_win]
  loser_str = '(' + str(final_game_loss) + ') ' + play_in_teams['West'][final_game_loss]
  new_row = {"Round":'Play-In', "Conf":"West", "Winner":winner_str, "Loser":loser_str, "Score":'1-0'}
  playoff_results = playoff_results.append(new_row, ignore_index=True)

  print(play_in_teams['West'][final_game_win] + ' makes the playoffs')
  print(play_in_teams['West'][final_game_loss] + ' is eliminated')
  input("Press enter to proceed: ")

  # remove loser
  del play_in_teams['West'][final_game_win]
  del play_in_teams['West'][final_game_loss]
  # save eight seed
  eight_seed = playoff_teams['West'].get(final_game_win)
  # remove from playoffs
  del playoff_teams['West'][final_game_win]
  del playoff_teams['West'][final_game_loss]
  # reset seven eight
  playoff_teams['West'][7] = seven_seed
  playoff_teams['West'][8] = eight_seed

  for team in playoff_teams['East']:
    east_ovr.at[east_ovr.index[east_ovr['TM'] == playoff_teams['East'][team]['Team']].tolist()[0], "Playoff App."] += 1

  for team in playoff_teams['West']:
    west_ovr.at[west_ovr.index[west_ovr['TM'] == playoff_teams['West'][team]['Team']].tolist()[0], "Playoff App."] += 1

  return playoff_results

##### Predict Playoff Game Function

In [21]:
def playoff_predict_matchup(team1, team2, nba_teams, matchup_teams, conf):
  rand_val = random.randint(0,nba_teams[conf][matchup_teams[conf][team1]['Team']] + nba_teams[conf][matchup_teams[conf][team2]['Team']])
  if rand_val < (nba_teams[conf][matchup_teams[conf][team1]['Team']] + 10):
    return team1, team2
  else:
    return team2, team1

##### First Round Function

In [22]:
def first_round(playoff_teams, nba_teams, east, west, playoff_results):
  # define series scores
  series_scores = {"East":{1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, "done":False}, "West":{1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, "done":False}}
  fin_series_count = 0
  one_eig_done = False
  four_five_done = False
  three_six_done = False
  two_sev_done = False
  # Eastern Conf. Round 1  Display
  print("Eastern Conf. Round 1")
  while series_scores['East']['done'] == False:
    # predict games
    if one_eig_done == False:
      print('(1) ' + playoff_teams['East'][1]['Team'] + ' vs. ' + playoff_teams['East'][8]['Team'] + ' (8) | Score: ' + str(series_scores['East'][1]) + '-' + str(series_scores['East'][8]))
      one_eig_win, one_eig_loss = playoff_predict_matchup(1, 8, nba_teams, playoff_teams, "East")
      series_scores['East'][one_eig_win] += 1
      if series_scores['East'][one_eig_win] == 4:
        fin_series_count += 1
        one_eig_done = True
    if four_five_done == False:
      print('(4) ' + playoff_teams['East'][4]['Team'] + ' vs. ' + playoff_teams['East'][5]['Team'] + ' (5) | Score: ' + str(series_scores['East'][4]) + '-' + str(series_scores['East'][5]))
      four_five_win, four_five_loss = playoff_predict_matchup(4, 5, nba_teams, playoff_teams, "East")
      series_scores['East'][four_five_win] += 1
      if series_scores['East'][four_five_win] == 4:
        fin_series_count += 1
        four_five_done = True
    if three_six_done == False:
      print('(3) ' + playoff_teams['East'][3]['Team'] + ' vs. ' + playoff_teams['East'][6]['Team'] + ' (6) | Score: ' + str(series_scores['East'][3]) + '-' + str(series_scores['East'][6]))
      three_six_win, three_six_loss = playoff_predict_matchup(3, 6, nba_teams, playoff_teams, "East")
      series_scores['East'][three_six_win] += 1
      if series_scores['East'][three_six_win] == 4:
        fin_series_count += 1
        three_six_done = True
    if two_sev_done == False:
      print('(2) ' + playoff_teams['East'][2]['Team'] + ' vs. ' + playoff_teams['East'][7]['Team'] + ' (7) | Score: ' + str(series_scores['East'][2]) + '-' + str(series_scores['East'][7]))
      two_sev_win, two_sev_loss = playoff_predict_matchup(2, 7, nba_teams, playoff_teams, "East")
      series_scores['East'][two_sev_win] += 1
      if series_scores['East'][two_sev_win] == 4:
        fin_series_count += 1
        two_sev_done = True

    input("Press enter to simulate game: ")
    print()

    # if all four done, finish
    if fin_series_count == 4:
      series_scores['East']['done'] = True

    # display winners
    print("Eastern Conf. Round 1 Results")
    if one_eig_done == True:
      try:
        print(playoff_teams['East'][one_eig_win]['Team'] + ' eliminates ' + playoff_teams['East'][one_eig_loss]['Team'] + ' | Score: ' + playoff_teams['East'][1]['Team'] + ' ' + str(series_scores['East'][1]) + '-' + str(series_scores['East'][8]) + ' ' + playoff_teams['East'][8]['Team'])
        winner_str = '(' + str(one_eig_win) + ') ' + playoff_teams['East'][one_eig_win]['Team']
        loser_str = '(' + str(one_eig_loss) + ') ' + playoff_teams['East'][one_eig_loss]['Team']
        score_str = str(series_scores['East'][one_eig_win]) + '-' + str(series_scores['East'][one_eig_loss])
        new_row = {"Round":'1st Rd.', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['East'][one_eig_loss]
      except:
        pass
    else:
      print(playoff_teams['East'][one_eig_win]['Team'] + ' defeats ' + playoff_teams['East'][one_eig_loss]['Team'] + '    | Score: ' + playoff_teams['East'][1]['Team'] + ' ' + str(series_scores['East'][1]) + '-' + str(series_scores['East'][8]) + ' ' + playoff_teams['East'][8]['Team'])

    if four_five_done == True:
      try:
        print(playoff_teams['East'][four_five_win]['Team'] + ' eliminates ' + playoff_teams['East'][four_five_loss]['Team'] + ' | Score: ' + playoff_teams['East'][4]['Team'] + ' ' + str(series_scores['East'][4]) + '-' + str(series_scores['East'][5]) + ' ' + playoff_teams['East'][5]['Team'])
        winner_str = '(' + str(four_five_win) + ') ' + playoff_teams['East'][four_five_win]['Team']
        loser_str = '(' + str(four_five_loss) + ') ' + playoff_teams['East'][four_five_loss]['Team']
        score_str = str(series_scores['East'][four_five_win]) + '-' + str(series_scores['East'][four_five_loss])
        new_row = {"Round":'1st Rd.', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['East'][four_five_loss]
      except:
        pass
    else:
      print(playoff_teams['East'][four_five_win]['Team'] + ' defeats ' + playoff_teams['East'][four_five_loss]['Team'] + '    | Score: ' + playoff_teams['East'][4]['Team'] + ' ' + str(series_scores['East'][4]) + '-' + str(series_scores['East'][5]) + ' ' + playoff_teams['East'][5]['Team'])

    if three_six_done == True:
      try:
        print(playoff_teams['East'][three_six_win]['Team'] + ' eliminates ' + playoff_teams['East'][three_six_loss]['Team'] + ' | Score: ' + playoff_teams['East'][3]['Team'] + ' ' + str(series_scores['East'][3]) + '-' + str(series_scores['East'][6]) + ' ' + playoff_teams['East'][6]['Team'])
        winner_str = '(' + str(three_six_win) + ') ' + playoff_teams['East'][three_six_win]['Team']
        loser_str = '(' + str(three_six_loss) + ') ' + playoff_teams['East'][three_six_loss]['Team']
        score_str = str(series_scores['East'][three_six_win]) + '-' + str(series_scores['East'][three_six_loss])
        new_row = {"Round":'1st Rd.', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['East'][three_six_loss]
      except:
        pass
    else:
      print(playoff_teams['East'][three_six_win]['Team'] + ' defeats ' + playoff_teams['East'][three_six_loss]['Team'] + '    | Score: ' + playoff_teams['East'][3]['Team'] + ' ' + str(series_scores['East'][3]) + '-' + str(series_scores['East'][6]) + ' ' + playoff_teams['East'][6]['Team'])

    if two_sev_done == True:
      try:
        print(playoff_teams['East'][two_sev_win]['Team'] + ' eliminates ' + playoff_teams['East'][two_sev_loss]['Team'] + ' | Score: ' + playoff_teams['East'][2]['Team'] + ' ' + str(series_scores['East'][2]) + '-' + str(series_scores['East'][7]) + ' ' + playoff_teams['East'][7]['Team'])
        winner_str = '(' + str(two_sev_win) + ') ' + playoff_teams['East'][two_sev_win]['Team']
        loser_str = '(' + str(two_sev_loss) + ') ' + playoff_teams['East'][two_sev_loss]['Team']
        score_str = str(series_scores['East'][two_sev_win]) + '-' + str(series_scores['East'][two_sev_loss])
        new_row = {"Round":'1st Rd.', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['East'][two_sev_loss]
      except:
        pass
    else:
      print(playoff_teams['East'][two_sev_win]['Team'] + ' defeats ' + playoff_teams['East'][two_sev_loss]['Team'] + '    | Score: ' + playoff_teams['East'][2]['Team'] + ' ' + str(series_scores['East'][2]) + '-' + str(series_scores['East'][7]) + ' ' + playoff_teams['East'][7]['Team'])
    input("Press enter to proceed: ")
    print()

  fin_series_count = 0
  one_eig_done = False
  four_five_done = False
  three_six_done = False
  two_sev_done = False
  # Western Conf. Round 1  Display
  print("Western Conf. Round 1")
  while series_scores['West']['done'] == False:
    # predict games
    if one_eig_done == False:
      print('(1) ' + playoff_teams['West'][1]['Team'] + ' vs. ' + playoff_teams['West'][8]['Team'] + ' (8) | Score: ' + str(series_scores['West'][1]) + '-' + str(series_scores['West'][8]))
      one_eig_win, one_eig_loss = playoff_predict_matchup(1, 8, nba_teams, playoff_teams, "West")
      series_scores['West'][one_eig_win] += 1
      if series_scores['West'][one_eig_win] == 4:
        fin_series_count += 1
        one_eig_done = True
    if four_five_done == False:
      print('(4) ' + playoff_teams['West'][4]['Team'] + ' vs. ' + playoff_teams['West'][5]['Team'] + ' (5) | Score: ' + str(series_scores['West'][4]) + '-' + str(series_scores['West'][5]))
      four_five_win, four_five_loss = playoff_predict_matchup(4, 5, nba_teams, playoff_teams, "West")
      series_scores['West'][four_five_win] += 1
      if series_scores['West'][four_five_win] == 4:
        fin_series_count += 1
        four_five_done = True
    if three_six_done == False:
      print('(3) ' + playoff_teams['West'][3]['Team'] + ' vs. ' + playoff_teams['West'][6]['Team'] + ' (6) | Score: ' + str(series_scores['West'][3]) + '-' + str(series_scores['West'][6]))
      three_six_win, three_six_loss = playoff_predict_matchup(3, 6, nba_teams, playoff_teams, "West")
      series_scores['West'][three_six_win] += 1
      if series_scores['West'][three_six_win] == 4:
        fin_series_count += 1
        three_six_done = True
    if two_sev_done == False:
      print('(2) ' + playoff_teams['West'][2]['Team'] + ' vs. ' + playoff_teams['West'][7]['Team'] + ' (7) | Score: ' + str(series_scores['West'][2]) + '-' + str(series_scores['West'][7]))
      two_sev_win, two_sev_loss = playoff_predict_matchup(2, 7, nba_teams, playoff_teams, "West")
      series_scores['West'][two_sev_win] += 1
      if series_scores['West'][two_sev_win] == 4:
        fin_series_count += 1
        two_sev_done = True

    input("Press enter to simulate game: ")
    print()

    # if all four done, finish
    if fin_series_count == 4:
      series_scores['West']['done'] = True

    # display winners
    print("Western Conf. Round 1 Results")
    if one_eig_done == True:
      try:
        print(playoff_teams['West'][one_eig_win]['Team'] + ' eliminates ' + playoff_teams['West'][one_eig_loss]['Team'] + ' | Score: ' + playoff_teams['West'][1]['Team'] + ' ' + str(series_scores['West'][1]) + '-' + str(series_scores['West'][8]) + ' ' + playoff_teams['West'][8]['Team'])
        winner_str = '(' + str(one_eig_win) + ') ' + playoff_teams['West'][one_eig_win]['Team']
        loser_str = '(' + str(one_eig_loss) + ') ' + playoff_teams['West'][one_eig_loss]['Team']
        score_str = str(series_scores['West'][one_eig_win]) + '-' + str(series_scores['West'][one_eig_loss])
        new_row = {"Round":'1st Rd.', "Conf":"West", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['West'][one_eig_loss]
      except:
        pass
    else:
      print(playoff_teams['West'][one_eig_win]['Team'] + ' defeats ' + playoff_teams['West'][one_eig_loss]['Team'] + '    | Score: ' + playoff_teams['West'][1]['Team'] + ' ' + str(series_scores['West'][1]) + '-' + str(series_scores['West'][8]) + ' ' + playoff_teams['West'][8]['Team'])

    if four_five_done == True:
      try:
        print(playoff_teams['West'][four_five_win]['Team'] + ' eliminates ' + playoff_teams['West'][four_five_loss]['Team'] + ' | Score: ' + playoff_teams['West'][4]['Team'] + ' ' + str(series_scores['West'][4]) + '-' + str(series_scores['West'][5]) + ' ' + playoff_teams['West'][5]['Team'])
        winner_str = '(' + str(four_five_win) + ') ' + playoff_teams['West'][four_five_win]['Team']
        loser_str = '(' + str(four_five_loss) + ') ' + playoff_teams['West'][four_five_loss]['Team']
        score_str = str(series_scores['West'][four_five_win]) + '-' + str(series_scores['West'][four_five_loss])
        new_row = {"Round":'1st Rd.', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['West'][four_five_loss]
      except:
        pass
    else:
      print(playoff_teams['West'][four_five_win]['Team'] + ' defeats ' + playoff_teams['West'][four_five_loss]['Team'] + '    | Score: ' + playoff_teams['West'][4]['Team'] + ' ' + str(series_scores['West'][4]) + '-' + str(series_scores['West'][5]) + ' ' + playoff_teams['West'][5]['Team'])

    if three_six_done == True:
      try:
        print(playoff_teams['West'][three_six_win]['Team'] + ' eliminates ' + playoff_teams['West'][three_six_loss]['Team'] + ' | Score: ' + playoff_teams['West'][3]['Team'] + ' ' + str(series_scores['West'][3]) + '-' + str(series_scores['West'][6]) + ' ' + playoff_teams['West'][6]['Team'])
        winner_str = '(' + str(three_six_win) + ') ' + playoff_teams['West'][three_six_win]['Team']
        loser_str = '(' + str(three_six_loss) + ') ' + playoff_teams['West'][three_six_loss]['Team']
        score_str = str(series_scores['West'][three_six_win]) + '-' + str(series_scores['West'][three_six_loss])
        new_row = {"Round":'1st Rd.', "Conf":"West", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['West'][three_six_loss]
      except:
        pass
    else:
      print(playoff_teams['West'][three_six_win]['Team'] + ' defeats ' + playoff_teams['West'][three_six_loss]['Team'] + '    | Score: ' + playoff_teams['West'][3]['Team'] + ' ' + str(series_scores['West'][3]) + '-' + str(series_scores['West'][6]) + ' ' + playoff_teams['West'][6]['Team'])

    if two_sev_done == True:
      try:
        print(playoff_teams['West'][two_sev_win]['Team'] + ' eliminates ' + playoff_teams['West'][two_sev_loss]['Team'] + ' | Score: ' + playoff_teams['West'][2]['Team'] + ' ' + str(series_scores['West'][2]) + '-' + str(series_scores['West'][7]) + ' ' + playoff_teams['West'][7]['Team'])
        winner_str = '(' + str(two_sev_win) + ') ' + playoff_teams['West'][two_sev_win]['Team']
        loser_str = '(' + str(two_sev_loss) + ') ' + playoff_teams['West'][two_sev_loss]['Team']
        score_str = str(series_scores['West'][two_sev_win]) + '-' + str(series_scores['West'][two_sev_loss])
        new_row = {"Round":'1st Rd.', "Conf":"West", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['West'][two_sev_loss]
      except:
        pass
    else:
      print(playoff_teams['West'][two_sev_win]['Team'] + ' defeats ' + playoff_teams['West'][two_sev_loss]['Team'] + '    | Score: ' + playoff_teams['West'][2]['Team'] + ' ' + str(series_scores['West'][2]) + '-' + str(series_scores['West'][7]) + ' ' + playoff_teams['West'][7]['Team'])
    input("Press enter to proceed: ")
    print()

  return playoff_results

##### Second Round Function

In [23]:
def second_round(playoff_teams, nba_teams, east, west, playoff_results):
  # set up the series scores
  series_scores = {"East":{}, "West":{}}
  for key in playoff_teams["East"].keys():
    series_scores["East"][key] = 0

  for key in playoff_teams["West"].keys():
    series_scores["West"][key] = 0

  # set up the matchups
  matchups = {"East":[[],[]], "West":[[],[]]}
  for seed in series_scores['East']:
    if seed == 1 or seed == 8:
      matchups["East"][0].append(seed)
    elif seed == 2 or seed == 7:
      matchups["East"][1].append(seed)
    elif seed == 3 or seed == 6:
      matchups["East"][1].append(seed)
    else:
      matchups["East"][0].append(seed)

  for seed in series_scores['West']:
    if seed == 1 or seed == 8:
      matchups["West"][0].append(seed)
    elif seed == 2 or seed == 7:
      matchups["West"][1].append(seed)
    elif seed == 3 or seed == 6:
      matchups["West"][1].append(seed)
    else:
      matchups["West"][0].append(seed)

  # add done option to series scores
  series_scores["East"]["done"] = False
  series_scores["West"]["done"] = False

  fin_series_count = 0
  match1_done = False
  match2_done = False
  # Eastern Conf. Round 2  Display
  print("Eastern Conf. Round 2")
  while series_scores['East']['done'] == False:
    # predict games
    if match1_done == False:
      print('(' + str(matchups['East'][0][0]) + ') ' + playoff_teams['East'][matchups['East'][0][0]]['Team'] + ' vs. ' + playoff_teams['East'][matchups['East'][0][1]]['Team'] + ' (' + str(matchups['East'][0][1]) + ') | Score: ' + str(series_scores['East'][matchups['East'][0][0]]) + '-' + str(series_scores['East'][matchups['East'][0][1]]))
      match1_win, match1_loss = playoff_predict_matchup(matchups['East'][0][0], matchups['East'][0][1], nba_teams, playoff_teams, "East")
      series_scores['East'][match1_win] += 1
      if series_scores['East'][match1_win] == 4:
        fin_series_count += 1
        match1_done = True
    if match2_done == False:
      print('(' + str(matchups['East'][1][0]) + ') ' + playoff_teams['East'][matchups['East'][1][0]]['Team'] + ' vs. ' + playoff_teams['East'][matchups['East'][1][1]]['Team'] + ' (' + str(matchups['East'][1][1]) + ') | Score: ' + str(series_scores['East'][matchups['East'][1][0]]) + '-' + str(series_scores['East'][matchups['East'][1][1]]))
      match2_win, match2_loss = playoff_predict_matchup(matchups['East'][1][0], matchups['East'][1][1], nba_teams, playoff_teams, "East")
      series_scores['East'][match2_win] += 1
      if series_scores['East'][match2_win] == 4:
        fin_series_count += 1
        match2_done = True

    input("Press enter to simulate game: ")
    print()

    # if all four done, finish
    if fin_series_count == 2:
      series_scores['East']['done'] = True

    # display winners
    print("Eastern Conf. Round 2 Results")
    if match1_done == True:
      try:
        print(playoff_teams['East'][match1_win]['Team'] + ' eliminates ' + playoff_teams['East'][match1_loss]['Team'] + ' | Score: ' + playoff_teams['East'][matchups['East'][0][0]]['Team'] + ' ' + str(series_scores['East'][matchups['East'][0][0]]) + '-' + str(series_scores['East'][matchups['East'][0][1]]) + ' ' + playoff_teams['East'][matchups['East'][0][1]]['Team'])
        winner_str = '(' + str(match1_win) + ') ' + playoff_teams['East'][match1_win]['Team']
        loser_str = '(' + str(match1_loss) + ') ' + playoff_teams['East'][match1_loss]['Team']
        score_str = str(series_scores['East'][match1_win]) + '-' + str(series_scores['East'][match1_loss])
        new_row = {"Round":'2nd Rd.', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['East'][match1_loss]
      except:
        pass
    else:
      print(playoff_teams['East'][match1_win]['Team'] + ' defeats ' + playoff_teams['East'][match1_loss]['Team'] + '    | Score: ' + playoff_teams['East'][matchups['East'][0][0]]['Team'] + ' ' + str(series_scores['East'][matchups['East'][0][0]]) + '-' + str(series_scores['East'][matchups['East'][0][1]]) + ' ' + playoff_teams['East'][matchups['East'][0][1]]['Team'])

    if match2_done == True:
      try:
        print(playoff_teams['East'][match2_win]['Team'] + ' eliminates ' + playoff_teams['East'][match2_loss]['Team'] + ' | Score: ' + playoff_teams['East'][matchups['East'][1][0]]['Team'] + ' ' + str(series_scores['East'][matchups['East'][1][0]]) + '-' + str(series_scores['East'][matchups['East'][1][1]]) + ' ' + playoff_teams['East'][matchups['East'][1][1]]['Team'])
        winner_str = '(' + str(match2_win) + ') ' + playoff_teams['East'][match2_win]['Team']
        loser_str = '(' + str(match2_loss) + ') ' + playoff_teams['East'][match2_loss]['Team']
        score_str = str(series_scores['East'][match2_win]) + '-' + str(series_scores['East'][match2_loss])
        new_row = {"Round":'2nd Rd.', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['East'][match2_loss]
      except:
        pass
    else:
      print(playoff_teams['East'][match2_win]['Team'] + ' defeats ' + playoff_teams['East'][match2_loss]['Team'] + '    | Score: ' + playoff_teams['East'][matchups['East'][1][0]]['Team'] + ' ' + str(series_scores['East'][matchups['East'][1][0]]) + '-' + str(series_scores['East'][matchups['East'][1][1]]) + ' ' + playoff_teams['East'][matchups['East'][1][1]]['Team'])

    input("Press enter to proceed: ")
    print()

  fin_series_count = 0
  match1_done = False
  match2_done = False
  # Western Conf. Round 2  Display
  print("Western Conf. Round 2")
  while series_scores['West']['done'] == False:
    # predict games
    if match1_done == False:
      print('(' + str(matchups['West'][0][0]) + ') ' + playoff_teams['West'][matchups['West'][0][0]]['Team'] + ' vs. ' + playoff_teams['West'][matchups['West'][0][1]]['Team'] + ' (' + str(matchups['West'][0][1]) + ') | Score: ' + str(series_scores['West'][matchups['West'][0][0]]) + '-' + str(series_scores['West'][matchups['West'][0][1]]))
      match1_win, match1_loss = playoff_predict_matchup(matchups['West'][0][0], matchups['West'][0][1], nba_teams, playoff_teams, "West")
      series_scores['West'][match1_win] += 1
      if series_scores['West'][match1_win] == 4:
        fin_series_count += 1
        match1_done = True
    if match2_done == False:
      print('(' + str(matchups['West'][1][0]) + ') ' + playoff_teams['West'][matchups['West'][1][0]]['Team'] + ' vs. ' + playoff_teams['West'][matchups['West'][1][1]]['Team'] + ' (' + str(matchups['West'][1][1]) + ') | Score: ' + str(series_scores['West'][matchups['West'][1][0]]) + '-' + str(series_scores['West'][matchups['West'][1][1]]))
      match2_win, match2_loss = playoff_predict_matchup(matchups['West'][1][0], matchups['West'][1][1], nba_teams, playoff_teams, "West")
      series_scores['West'][match2_win] += 1
      if series_scores['West'][match2_win] == 4:
        fin_series_count += 1
        match2_done = True

    input("Press enter to simulate game: ")
    print()

    # if all four done, finish
    if fin_series_count == 2:
      series_scores['West']['done'] = True

    # display winners
    print("Western Conf. Round 2 Results")
    if match1_done == True:
      try:
        print(playoff_teams['West'][match1_win]['Team'] + ' eliminates ' + playoff_teams['West'][match1_loss]['Team'] + ' | Score: ' + playoff_teams['West'][matchups['West'][0][0]]['Team'] + ' ' + str(series_scores['West'][matchups['West'][0][0]]) + '-' + str(series_scores['West'][matchups['West'][0][1]]) + ' ' + playoff_teams['West'][matchups['West'][0][1]]['Team'])
        winner_str = '(' + str(match1_win) + ') ' + playoff_teams['West'][match1_win]['Team']
        loser_str = '(' + str(match1_loss) + ') ' + playoff_teams['West'][match1_loss]['Team']
        score_str = str(series_scores['West'][match1_win]) + '-' + str(series_scores['West'][match1_loss])
        new_row = {"Round":'2nd Rd.', "Conf":"West", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['West'][match1_loss]
      except:
        pass
    else:
      print(playoff_teams['West'][match1_win]['Team'] + ' defeats ' + playoff_teams['West'][match1_loss]['Team'] + '    | Score: ' + playoff_teams['West'][matchups['West'][0][0]]['Team'] + ' ' + str(series_scores['West'][matchups['West'][0][0]]) + '-' + str(series_scores['West'][matchups['West'][0][1]]) + ' ' + playoff_teams['West'][matchups['West'][0][1]]['Team'])

    if match2_done == True:
      try:
        print(playoff_teams['West'][match2_win]['Team'] + ' eliminates ' + playoff_teams['West'][match2_loss]['Team'] + ' | Score: ' + playoff_teams['West'][matchups['West'][1][0]]['Team'] + ' ' + str(series_scores['West'][matchups['West'][1][0]]) + '-' + str(series_scores['West'][matchups['West'][1][1]]) + ' ' + playoff_teams['West'][matchups['West'][1][1]]['Team'])
        winner_str = '(' + str(match2_win) + ') ' + playoff_teams['West'][match2_win]['Team']
        loser_str = '(' + str(match2_loss) + ') ' + playoff_teams['West'][match2_loss]['Team']
        score_str = str(series_scores['West'][match2_win]) + '-' + str(series_scores['West'][match2_loss])
        new_row = {"Round":'2nd Rd.', "Conf":"West", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['West'][match2_loss]
      except:
        pass
    else:
      print(playoff_teams['West'][match2_win]['Team'] + ' defeats ' + playoff_teams['West'][match2_loss]['Team'] + '    | Score: ' + playoff_teams['West'][matchups['West'][1][0]]['Team'] + ' ' + str(series_scores['West'][matchups['West'][1][0]]) + '-' + str(series_scores['West'][matchups['West'][1][1]]) + ' ' + playoff_teams['West'][matchups['West'][1][1]]['Team'])

    input("Press enter to proceed: ")
    print()

  return playoff_results

##### Conf. Finals Function

In [24]:
def conf_finals(playoff_teams, nba_teams, east, west, playoff_results):
  # set up the series scores
  series_scores = {"East":{}, "West":{}}
  for key in playoff_teams["East"].keys():
    series_scores["East"][key] = 0

  for key in playoff_teams["West"].keys():
    series_scores["West"][key] = 0

  # set up the matchups
  matchups = {"East":[], "West":[]}
  for seed in series_scores['East']:
    matchups["East"].append(seed)

  for seed in series_scores['West']:
    matchups["West"].append(seed)

  # add done option to series scores
  series_scores["East"]["done"] = False
  series_scores["West"]["done"] = False

  fin_series_count = 0
  match1_done = False
  # Eastern Conf. Finals Display
  print("Eastern Conf. Finals")
  while series_scores['East']['done'] == False:
    # predict games
    if match1_done == False:
      print('(' + str(matchups['East'][0]) + ') ' + playoff_teams['East'][matchups['East'][0]]['Team'] + ' vs. ' + playoff_teams['East'][matchups['East'][1]]['Team'] + ' (' + str(matchups['East'][1]) + ') | Score: ' + str(series_scores['East'][matchups['East'][0]]) + '-' + str(series_scores['East'][matchups['East'][1]]))
      match1_win, match1_loss = playoff_predict_matchup(matchups['East'][0], matchups['East'][1], nba_teams, playoff_teams, "East")
      series_scores['East'][match1_win] += 1
      if series_scores['East'][match1_win] == 4:
        fin_series_count += 1
        match1_done = True

    input("Press enter to simulate game: ")
    print()

    # if all four done, finish
    if fin_series_count == 1:
      series_scores['East']['done'] = True

    # display winners
    print("Eastern Conf. Finals Results")
    if match1_done == True:
      try:
        print(playoff_teams['East'][match1_win]['Team'] + ' eliminates ' + playoff_teams['East'][match1_loss]['Team'] + ' | Score: ' + playoff_teams['East'][matchups['East'][0]]['Team'] + ' ' + str(series_scores['East'][matchups['East'][0]]) + '-' + str(series_scores['East'][matchups['East'][1]]) + ' ' + playoff_teams['East'][matchups['East'][1]]['Team'])
        winner_str = '(' + str(match1_win) + ') ' + playoff_teams['East'][match1_win]['Team']
        loser_str = '(' + str(match1_loss) + ') ' + playoff_teams['East'][match1_loss]['Team']
        score_str = str(series_scores['East'][match1_win]) + '-' + str(series_scores['East'][match1_loss])
        new_row = {"Round":'Conf. Finals', "Conf":"East", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['East'][match1_loss]
      except:
        pass
    else:
      print(playoff_teams['East'][match1_win]['Team'] + ' defeats ' + playoff_teams['East'][match1_loss]['Team'] + '    | Score: ' + playoff_teams['East'][matchups['East'][0]]['Team'] + ' ' + str(series_scores['East'][matchups['East'][0]]) + '-' + str(series_scores['East'][matchups['East'][1]]) + ' ' + playoff_teams['East'][matchups['East'][1]]['Team'])

    input("Press enter to proceed: ")
    print()

  fin_series_count = 0
  match1_done = False
  # Western Conf. Finals Display
  print("Western Conf. Finals")
  while series_scores['West']['done'] == False:
    # predict games
    if match1_done == False:
      print('(' + str(matchups['West'][0]) + ') ' + playoff_teams['West'][matchups['West'][0]]['Team'] + ' vs. ' + playoff_teams['West'][matchups['West'][1]]['Team'] + ' (' + str(matchups['West'][1]) + ') | Score: ' + str(series_scores['West'][matchups['West'][0]]) + '-' + str(series_scores['West'][matchups['West'][1]]))
      match1_win, match1_loss = playoff_predict_matchup(matchups['West'][0], matchups['West'][1], nba_teams, playoff_teams, "West")
      series_scores['West'][match1_win] += 1
      if series_scores['West'][match1_win] == 4:
        fin_series_count += 1
        match1_done = True

    input("Press enter to simulate game: ")
    print()

    # if all four done, finish
    if fin_series_count == 1:
      series_scores['West']['done'] = True

    # display winners
    print("Western Conf. Finals Results")
    if match1_done == True:
      try:
        print(playoff_teams['West'][match1_win]['Team'] + ' eliminates ' + playoff_teams['West'][match1_loss]['Team'] + ' | Score: ' + playoff_teams['West'][matchups['West'][0]]['Team'] + ' ' + str(series_scores['West'][matchups['West'][0]]) + '-' + str(series_scores['West'][matchups['West'][1]]) + ' ' + playoff_teams['West'][matchups['West'][1]]['Team'])
        winner_str = '(' + str(match1_win) + ') ' + playoff_teams['West'][match1_win]['Team']
        loser_str = '(' + str(match1_loss) + ') ' + playoff_teams['West'][match1_loss]['Team']
        score_str = str(series_scores['West'][match1_win]) + '-' + str(series_scores['West'][match1_loss])
        new_row = {"Round":'Conf. Finals', "Conf":"West", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams['West'][match1_loss]
      except:
        pass
    else:
      print(playoff_teams['West'][match1_win]['Team'] + ' defeats ' + playoff_teams['West'][match1_loss]['Team'] + '    | Score: ' + playoff_teams['West'][matchups['West'][0]]['Team'] + ' ' + str(series_scores['West'][matchups['West'][0]]) + '-' + str(series_scores['West'][matchups['West'][1]]) + ' ' + playoff_teams['West'][matchups['West'][1]]['Team'])

    input("Press enter to proceed: ")
    print()

  return playoff_results

##### Predict Finals Game Function

In [25]:
def finals_predict_matchup(team1, team2, nba_teams, matchup_teams):
  rand_val = random.randint(0,nba_teams["East"][matchup_teams["East"][team1]['Team']] + nba_teams["West"][matchup_teams["West"][team2]['Team']])
  if rand_val < nba_teams["East"][matchup_teams["East"][team1]['Team']]:
    return "East", "West"
  else:
    return "West", "East"

##### Finals Function

In [26]:
def finals(playoff_teams, nba_teams, east, west, east_ovr, west_ovr, playoff_results):

  # set up the series scores
  series_scores = {"East":0, "West":0, "done":False}

  # set up the matchups
  matchups = []
  for seed in playoff_teams['East'].keys():
    matchups.append(seed)
    east_ovr.at[east_ovr.index[east_ovr['TM'] == playoff_teams['East'][seed]['Team']].tolist()[0], "Champ App."] += 1

  for seed in playoff_teams['West'].keys():
    matchups.append(seed)
    west_ovr.at[west_ovr.index[west_ovr['TM'] == playoff_teams['West'][seed]['Team']].tolist()[0], "Champ App."] += 1

  fin_series_count = 0
  match1_done = False
  # NBA Finals Display
  print("NBA Finals")
  while series_scores['done'] == False:
    # predict games
    if match1_done == False:
      print('(' + str(matchups[0]) + ') ' + playoff_teams['East'][matchups[0]]['Team'] + ' vs. ' + playoff_teams['West'][matchups[1]]['Team'] + ' (' + str(matchups[1]) + ') | Score: ' + str(series_scores['East']) + '-' + str(series_scores['West']))
      match1_win, match1_loss = finals_predict_matchup(matchups[0], matchups[1], nba_teams, playoff_teams)
      series_scores[match1_win] += 1
      if match1_win == "East":
        winner = matchups[0]
        loser = matchups[1]
      else:
        winner = matchups[1]
        loser = matchups[0]

      if series_scores[match1_win] == 4:
        fin_series_count += 1
        match1_done = True

    input("Press enter to simulate game: ")
    print()

    # if all four done, finish
    if fin_series_count == 1:
      series_scores['done'] = True

    # display winners
    print("NBA Finals Results")
    if match1_done == True:
      try:
        print(playoff_teams[match1_win][winner]['Team'] + ' eliminates ' + playoff_teams[match1_loss][loser]['Team'] + ' | Score: ' + playoff_teams['East'][matchups[0]]['Team'] + ' ' + str(series_scores['East']) + '-' + str(series_scores['West']) + ' ' + playoff_teams['West'][matchups[1]]['Team'])
        winner_str = '(' + str(winner) + ') ' + playoff_teams[match1_win][winner]['Team']
        loser_str = '(' + str(loser) + ') ' + playoff_teams[match1_loss][loser]['Team']
        if match1_loss == 'West':
          score_str = str(series_scores['East']) + '-' + str(series_scores['West'])
        else:
          score_str = str(series_scores['West']) + '-' + str(series_scores['East'])

        new_row = {"Round":'NBA Finals', "Conf":"N/A", "Winner":winner_str, "Loser":loser_str, "Score":score_str}
        playoff_results = playoff_results.append(new_row, ignore_index=True)

        del playoff_teams[loser][match1_loss]
      except:
        pass
    else:
      print(playoff_teams[match1_win][winner]['Team'] + ' defeats ' + playoff_teams[match1_loss][loser]['Team'] + '    | Score: ' + playoff_teams['East'][matchups[0]]['Team'] + ' ' + str(series_scores['East']) + '-' + str(series_scores['West']) + ' ' + playoff_teams['West'][matchups[1]]['Team'])

    input("Press enter to proceed: ")
    print()

  return playoff_results

### Run the Playoffs

Run the play in.

In [27]:
playoff_results = play_in(playoff_teams, nba_teams, east, west, east_ovr, west_ovr, playoff_results)

Eastern Conf. Play-In Tournament
(7) NYK vs. DET (8)
(9) IND vs. BKN (10)
Press enter to simulate: 

Eastern Conf. Play-In Rd. 1 Results
DET defeats NYK
BKN defeats IND
DET makes the playoffs
IND is eliminated
Press enter to proceed: 

Eastern Conf. Play-In Tournament Final Game
(7) NYK vs. BKN (10)
Press enter to simulate: 

Eastern Conf. Play-In Rd. 2 Results
NYK defeats BKN
NYK makes the playoffs
BKN is eliminated
Press enter to proceed: 

Western Conf. Play-In Tournament
(7) GSW vs. NOP (8)
(9) OKC vs. LAL (10)
Press enter to simulate: 

Western Conf. Play-In Rd. 1 Results
GSW defeats NOP
OKC defeats LAL
GSW makes the playoffs
LAL is eliminated
Press enter to proceed: 

Western Conf. Play-In Tournament Final Game
(8) NOP vs. OKC (9)
Press enter to simulate: 

Western Conf. Play-In Rd. 2 Results
OKC defeats NOP
OKC makes the playoffs
NOP is eliminated
Press enter to proceed: 


Run the first round.

In [28]:
playoff_results = first_round(playoff_teams, nba_teams, east, west, playoff_results)

Eastern Conf. Round 1
(1) MIL vs. NYK (8) | Score: 0-0
(4) MIA vs. PHI (5) | Score: 0-0
(3) BOS vs. CHI (6) | Score: 0-0
(2) CLE vs. DET (7) | Score: 0-0
Press enter to simulate game: 

Eastern Conf. Round 1 Results
NYK defeats MIL    | Score: MIL 0-1 NYK
MIA defeats PHI    | Score: MIA 1-0 PHI
CHI defeats BOS    | Score: BOS 0-1 CHI
CLE defeats DET    | Score: CLE 1-0 DET
Press enter to proceed: 

(1) MIL vs. NYK (8) | Score: 0-1
(4) MIA vs. PHI (5) | Score: 1-0
(3) BOS vs. CHI (6) | Score: 0-1
(2) CLE vs. DET (7) | Score: 1-0
Press enter to simulate game: 

Eastern Conf. Round 1 Results
NYK defeats MIL    | Score: MIL 0-2 NYK
MIA defeats PHI    | Score: MIA 2-0 PHI
CHI defeats BOS    | Score: BOS 0-2 CHI
CLE defeats DET    | Score: CLE 2-0 DET
Press enter to proceed: 

(1) MIL vs. NYK (8) | Score: 0-2
(4) MIA vs. PHI (5) | Score: 2-0
(3) BOS vs. CHI (6) | Score: 0-2
(2) CLE vs. DET (7) | Score: 2-0
Press enter to simulate game: 

Eastern Conf. Round 1 Results
MIL defeats NYK    | Sco

Run the second round.

In [29]:
playoff_results = second_round(playoff_teams, nba_teams, east, west, playoff_results)

Eastern Conf. Round 2
(1) MIL vs. PHI (5) | Score: 0-0
(6) CHI vs. DET (7) | Score: 0-0
Press enter to simulate game: 

Eastern Conf. Round 2 Results
PHI defeats MIL    | Score: MIL 0-1 PHI
CHI defeats DET    | Score: CHI 1-0 DET
Press enter to proceed: 

(1) MIL vs. PHI (5) | Score: 0-1
(6) CHI vs. DET (7) | Score: 1-0
Press enter to simulate game: 

Eastern Conf. Round 2 Results
MIL defeats PHI    | Score: MIL 1-1 PHI
CHI defeats DET    | Score: CHI 2-0 DET
Press enter to proceed: 

(1) MIL vs. PHI (5) | Score: 1-1
(6) CHI vs. DET (7) | Score: 2-0
Press enter to simulate game: 

Eastern Conf. Round 2 Results
PHI defeats MIL    | Score: MIL 1-2 PHI
CHI defeats DET    | Score: CHI 3-0 DET
Press enter to proceed: 

(1) MIL vs. PHI (5) | Score: 1-2
(6) CHI vs. DET (7) | Score: 3-0
Press enter to simulate game: 

Eastern Conf. Round 2 Results
MIL defeats PHI    | Score: MIL 2-2 PHI
DET defeats CHI    | Score: CHI 3-1 DET
Press enter to proceed: 

(1) MIL vs. PHI (5) | Score: 2-2
(6) CHI v

Run the conference finals.

In [30]:
playoff_results = conf_finals(playoff_teams, nba_teams, east, west, playoff_results)

Eastern Conf. Finals
(1) MIL vs. CHI (6) | Score: 0-0
Press enter to simulate game: 

Eastern Conf. Finals Results
CHI defeats MIL    | Score: MIL 0-1 CHI
Press enter to proceed: 

(1) MIL vs. CHI (6) | Score: 0-1
Press enter to simulate game: 

Eastern Conf. Finals Results
CHI defeats MIL    | Score: MIL 0-2 CHI
Press enter to proceed: 

(1) MIL vs. CHI (6) | Score: 0-2
Press enter to simulate game: 

Eastern Conf. Finals Results
MIL defeats CHI    | Score: MIL 1-2 CHI
Press enter to proceed: 

(1) MIL vs. CHI (6) | Score: 1-2
Press enter to simulate game: 

Eastern Conf. Finals Results
CHI defeats MIL    | Score: MIL 1-3 CHI
Press enter to proceed: 

(1) MIL vs. CHI (6) | Score: 1-3
Press enter to simulate game: 

Eastern Conf. Finals Results
MIL defeats CHI    | Score: MIL 2-3 CHI
Press enter to proceed: 

(1) MIL vs. CHI (6) | Score: 2-3
Press enter to simulate game: 

Eastern Conf. Finals Results
MIL defeats CHI    | Score: MIL 3-3 CHI
Press enter to proceed: 

(1) MIL vs. CHI (6)

Run the finals.

In [31]:
playoff_results = finals(playoff_teams, nba_teams, east, west, east_ovr, west_ovr, playoff_results)

NBA Finals
(1) MIL vs. PHX (2) | Score: 0-0
Press enter to simulate game: 

NBA Finals Results
PHX defeats MIL    | Score: MIL 0-1 PHX
Press enter to proceed: 

(1) MIL vs. PHX (2) | Score: 0-1
Press enter to simulate game: 

NBA Finals Results
PHX defeats MIL    | Score: MIL 0-2 PHX
Press enter to proceed: 

(1) MIL vs. PHX (2) | Score: 0-2
Press enter to simulate game: 

NBA Finals Results
PHX defeats MIL    | Score: MIL 0-3 PHX
Press enter to proceed: 

(1) MIL vs. PHX (2) | Score: 0-3
Press enter to simulate game: 

NBA Finals Results
PHX eliminates MIL | Score: MIL 0-4 PHX
Press enter to proceed: 



## Offseason

#### Adjust the League History

In [32]:
try:
  east_ovr.at[east_ovr.index[east_ovr['TM'] == playoff_results.loc[[20]]['Winner'].tolist()[0][4:]].tolist()[0], "Champ"] += 1
except:
  west_ovr.at[west_ovr.index[west_ovr['TM'] == playoff_results.loc[[20]]['Winner'].tolist()[0][4:]].tolist()[0], "Champ"] += 1

playoff_results.to_csv('playoffs' + str(season) + '.csv')

#### Run the Offseason Function

Define the normal distribution of offseason point changes.

In [33]:
import numpy as np
normal_dist = np.random.normal(loc=0, scale=5, size=1000).tolist()
for i in range(len(normal_dist)):
  normal_dist[i] = round(normal_dist[i])

In [34]:
def run_offseason(nba_teams, normal_dist, longevity):
  for team in nba_teams['East']:
    rand_change = random.randint(0,len(normal_dist)-1)
    nba_teams["East"][team] = nba_teams["East"][team] + normal_dist[rand_change] + longevity["East"][team]
    if east_ovr.index[east_ovr['TM'] == team] > 12:
      longevity["East"][team] += 2
    elif east_ovr.index[east_ovr['TM'] == team] <= 12 and east_ovr.index[east_ovr['TM'] == team] > 9:
      longevity["East"][team] += 1
    elif east_ovr.index[east_ovr['TM'] == team] <= 6 and east_ovr.index[east_ovr['TM'] == team] > 3:
      longevity["East"][team] -= 1
    elif east_ovr.index[east_ovr['TM'] == team] <= 3:
      longevity["East"][team] -= 2


  for team in nba_teams['West']:
    rand_change = random.randint(0,len(normal_dist)-1)
    nba_teams["West"][team] = nba_teams["West"][team] + normal_dist[rand_change] + longevity["West"][team]
    if west_ovr.index[west_ovr['TM'] == team] > 12:
      longevity["West"][team] += 2
    elif west_ovr.index[west_ovr['TM'] == team] <= 12 and west_ovr.index[west_ovr['TM'] == team] > 9:
      longevity["West"][team] += 1
    elif west_ovr.index[west_ovr['TM'] == team] <= 6 and west_ovr.index[west_ovr['TM'] == team] > 3:
      longevity["West"][team] -= 1
    elif west_ovr.index[west_ovr['TM'] == team] <= 3:
      longevity["West"][team] -= 2

  # adjustment to keep teams from getting overpowered
  for team in nba_teams['East']:
    if nba_teams['East'][team] < 32:
      nba_teams['East'][team] = 32
    elif nba_teams['East'][team] > 68:
      nba_teams['East'][team] = 68

  for team in nba_teams['West']:
    if nba_teams['West'][team] < 32:
      nba_teams['West'][team] = 32
    elif nba_teams['West'][team] > 68:
      nba_teams['West'][team] = 68

  for team in nba_teams['East']:
    if longevity['East'][team] < -5:
      longevity['East'][team] = -5
    elif longevity['East'][team] > 5:
      longevity['East'][team] = 5

  for team in nba_teams['West']:
    if longevity['West'][team] < -5:
      longevity['West'][team] = -5
    elif longevity['West'][team] > 5:
      longevity['West'][team] = 5

  east_sum = 0
  for team in nba_teams['East']:
    east_sum += nba_teams['East'][team]

  west_sum = 0
  for team in nba_teams['West']:
    west_sum += nba_teams['West'][team]

  lg_avg = (east_sum + west_sum) / 30
  print(nba_teams)
  print(lg_avg)
  if lg_avg > 51 or lg_avg < 49:
    for team in nba_teams['East']:
      nba_teams['East'][team] = round((nba_teams['East'][team]*(50/lg_avg)))

    for team in nba_teams['West']:
      nba_teams['West'][team] = round((nba_teams['West'][team]*(50/lg_avg)))

  print(nba_teams)

  return nba_teams


nba_teams = run_offseason(nba_teams, normal_dist, longevity)
season += 1

{'East': {'BOS': 68, 'CLE': 68, 'MIL': 68, 'PHI': 64, 'BKN': 52, 'NYK': 48, 'ATL': 48, 'IND': 45, 'TOR': 33, 'MIA': 54, 'WAS': 37, 'ORL': 51, 'CHI': 50, 'CHA': 32, 'DET': 62}, 'West': {'DAL': 60, 'SAC': 59, 'MEM': 63, 'DEN': 68, 'MIN': 52, 'PHX': 68, 'LAC': 46, 'GSW': 54, 'NOP': 49, 'UTA': 44, 'POR': 37, 'OKC': 66, 'LAL': 51, 'SAS': 32, 'HOU': 46}}
52.5
{'East': {'BOS': 65, 'CLE': 65, 'MIL': 65, 'PHI': 61, 'BKN': 50, 'NYK': 46, 'ATL': 46, 'IND': 43, 'TOR': 31, 'MIA': 51, 'WAS': 35, 'ORL': 49, 'CHI': 48, 'CHA': 30, 'DET': 59}, 'West': {'DAL': 57, 'SAC': 56, 'MEM': 60, 'DEN': 65, 'MIN': 50, 'PHX': 65, 'LAC': 44, 'GSW': 51, 'NOP': 47, 'UTA': 42, 'POR': 35, 'OKC': 63, 'LAL': 49, 'SAS': 30, 'HOU': 44}}
